In [5]:
# from rest_framework import status, viewsets
# from rest_framework.response import Response
# from rest_framework.decorators import action
# from django.contrib.auth import authenticate
import numpy as np
import pandas as pd
# import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import os
from django.conf import settings
import tensorflow as tf
from scipy.stats import norm 
class MLModelService:
    def __init__(self):
        self.model = load_model('drought_prediction_model.keras')
        self.df = pd.read_csv('Data_set.csv')
        self.daily_stats = pd.read_csv('daily_statistics.csv')
    
    
    def make_prediction(self):
        df = self.df
        df['Date'] = pd.to_datetime(df['Date'])
        model = self.model

        

        # Select relevant columns
        dff = df[['Date', 'MaxT', 'MinT', 'RH1', 'RH2', 'Wind', 'Rain', 'Evap', 'Radiation', 'Lat', 'Lon', 'Cum_Rain']]

        # Fix specific data point
        X = dff.drop('Date', axis=1)

        # Scale the features
        scaler = MinMaxScaler()
        scaler.fit(X)

        inp = X.tail(7).to_numpy()
        inp_norm = scaler.transform(inp)
        serri = inp_norm.reshape(1, 7, 11)

        # Predict drought using the model
        predictions = []
        spei_one_day = []
        # Output predictions and SPEI for each date
        start_date = pd.to_datetime('2018-09-01')
        end_date = pd.to_datetime('2018-09-08')
        date_range = pd.date_range(start_date, end_date, freq='D')


        current_date = date_range[0]
        day_of_year = current_date.dayofyear
        predict = model.predict(serri)
        pre_inv = scaler.inverse_transform(predict)
        predictions.append(pre_inv)
        precipitation_day = predictions[0][0][5]  # Assuming 'Rain' is the 6th column
        evaporation_day = predictions[0][0][6] 
        spei_value = self.calculate_spei_one_day(day_of_year, precipitation_day, evaporation_day, self.daily_stats)
        spei_one_day.append(spei_value)
        print(len(spei_one_day))

        # Calculate SPEI for one day


        for i in range(7):
            # Get day of year for the current prediction dateas
            current_date = date_range[i+1]
            day_of_year = current_date.dayofyear

            #predict
            predict = model.predict(serri)
            pre_inv = scaler.inverse_transform(predict)

            # Calculate SPEI for the new data point
            predictions.append(pre_inv)
            np.concatenate((inp[1:], pre_inv), axis=0)
            precipitation_day = predictions[0][0][5]  # Assuming 'Rain' is the 6th column
            evaporation_day = predictions[0][0][6]    # Assuming 'Evap' is the 7th column
            spei_value = self.calculate_spei_one_day(day_of_year, precipitation_day, evaporation_day, self.daily_stats)
            spei_one_day.append(spei_value)
            print(len(spei_one_day))


        p_date = {}

        for i in range(len(predictions)):
            p_date[date_range[i]] = {"prediction": predictions[i], "spei": spei_one_day[i]}

        for date, values in p_date.items():
            print(date, values)

        # Read daily statistics from CSV file
        daily_stats = pd.read_csv('daily_statistics.csv')

    def calculate_spei_one_day(self, day_of_year, precipitation, evaporation, stats):
        ref_precip_mean = stats.loc[stats['DayOfYear'] == day_of_year, 'Precip_Mean'].values[0]
        ref_precip_std = stats.loc[stats['DayOfYear'] == day_of_year, 'Precip_Std'].values[0]
        ref_evap_mean = stats.loc[stats['DayOfYear'] == day_of_year, 'Evap_Mean'].values[0]
        ref_evap_std = stats.loc[stats['DayOfYear'] == day_of_year, 'Evap_Std'].values[0]

        # Compute standardized anomalies
        precip_anomaly = (precipitation - ref_precip_mean) / ref_precip_std
        evap_anomaly = (evaporation - ref_evap_mean) / ref_evap_std

        # Combine the anomalies
        D_anomaly = precip_anomaly - evap_anomaly

        # Convert the anomaly to a standardized SPEI value using the inverse normal CDF (probit function)
        spei_value = norm.ppf(norm.cdf(D_anomaly))

        return spei_value

class DataProcessingService:
        pass
    
model = MLModelService()
model.make_prediction()



# class AuthenticationService:
#     @action(detail=False, methods=['post'])
#     def authenticate(self, request):
#         username = request.data.get('username')
#         password = request.data.get('password')
#         user = authenticate(username=username, password=password)
#         if user is not None:
#             return Response({'status': 'Authenticated'}, status=status.HTTP_200_OK)
#         return Response({'status': 'Unauthorized'}, status=status.HTTP_401_UNAUTHORIZED)

#     @classmethod
#     def get_extra_actions(cls):
#         return []


C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
8
2018-09-01 00:00:00 {'prediction': array([[  30.790573 ,   21.366589 ,   90.01861  ,   54.756695 ,
           8.551288 ,    2.0127497,    4.4324546,   18.75979  ,
          17.508202 ,   78.27264  , 1543.078    ]], dtype=float32), 'spei': -0.3750534555839103}
2018-09-02 00:00:00 {'prediction': array([[  30.790573 ,   21.366589 ,   90.01861  ,   54.756695 ,
           8.551288 ,    2.0127497,    4.4324546,   18.75979  ,
          17.508202 ,   78.27264  , 1543.078    ]], dtype=float32), 'spei': -0.17677682657990035}
2018-09-03 00:00:00 {'prediction': array([[  30.790573 ,   21.366589 ,   90.01861  ,   54.756695 ,
           8.551288 ,    2.0127497,    4.4324546,   18.7597

In [2]:
pip show pandas

Name: pandas
Version: 1.5.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: C:\Users\Hp\anaconda3\Lib\site-packages
Requires: numpy, numpy, python-dateutil, pytz
Required-by: bokeh, datashader, holoviews, hvplot, panel, seaborn, statsmodels, xarray
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ab/63/966db1321a0ad55df1d1fe51505d2cdae191b84c907974873817b0a6e849/pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB 640.0 kB/s eta 0:00:19
   ---------------------------------------- 0.0/11.6 MB 640.0 kB/s eta 0:00:19
   ---------------------------------------- 0.1/11.6 MB 465.5 kB/s eta 0:00:25
   ---------------------------------------- 0.1/11.6 MB 581.0 kB/s eta 0:00:20
   ---------------------------------------- 0.1/11.6 MB 554.9 kB/s eta 0:00:21
    --------------------------------------- 0.2/11.6 MB 653.6 kB/s eta 0:00:18
    --------------------------------------- 0.2/11.6 MB 808.4 kB/s eta 0:00:15
   - -------------------------------------- 0.3/11.6 MB 905.4 kB/s eta 0:00:13
   - -------------------------------------- 0.4/11.6 MB 1.1 MB/s eta 0:00:11
   - ----------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Hp\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
pip show pandas

Name: pandas
Version: 2.2.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License

Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
All rights reserved.

Copyright (c) 2011-2023, Open source contributors.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be u